## 構成ファイルの読み込み

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using System.IO;
using Microsoft.Extensions.Configuration;

var config = new ConfigurationBuilder()
    .AddJsonFile( Path.Combine(Environment.CurrentDirectory, "appsettings.json"))
    .Build();
var endpoint = config["AOAI_ENDPOINT"];
var modelDeploy = config["MODEL_DEPLOY_NAME"];


Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

## Semantic Kernel および 周辺ライブラリを読み込み

本記事執筆時点（2024 年 11 月 12 日）ではライブラリのアップデートが頻繁なためバージョンを 1.28.0 に固定。
今後のアップデートでは動作しなくなる可能性もあるため注意。

- [nuget: Semantic Kernel](https://www.nuget.org/packages/Microsoft.SemanticKernel)


In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.28.0"
#r "nuget: Azure.Identity"
#r "nuget: Microsoft.Extensions.DependencyInjection"
#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Microsoft.Extensions.Logging.Console"


Installed Packages Azure.Identity, 1.13.1 Microsoft.Extensions.DependencyInjection, 8.0.1 Microsoft.Extensions.Logging, 8.0.1 Microsoft.Extensions.Logging.Console, 8.0.1 Microsoft.SemanticKernel, 1.28.0

### ネイティブ プラグイン を定義

サンプルのため簡単なものを２つだけ定義

In [3]:
using Microsoft.SemanticKernel;
using Microsoft.Extensions.Logging;
using System.ComponentModel;

public class MyPlugin(ILogger<MyPlugin> logger)
{
    [KernelFunction("get_today")]
    [Description("今日の日付を取得します")]
    [return: Description("今日の日付")]
    public DateTimeOffset GetToday()
    {
        logger.LogInformation("getting today");
        return DateTimeOffset.UtcNow;
    }

    [KernelFunction("get_weather")]
    [Description("指定の日付の天気を回答します。過去日付の場合は履歴を、未来日付の場合は予報をこたえます。")]
    [return: Description("天気")]
    public string GetWeater(DateTimeOffset date)
    {
        logger.LogInformation("getting weather on {date}", date);
        var weather = new string[]{"晴れ", "曇り", "雨", "雪"};
        return weather[new Random().Next(0, weather.Length)];
    }
}

### Kernel にプラグインを読み込んで組み立てる

In [4]:
using Microsoft.Extensions.DependencyInjection;
using Azure.Identity;

// Azure OpenAI
var builder = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(modelDeploy, endpoint, new DefaultAzureCredential());

// ログを標準出力に
builder.Services.AddLogging( lb => {
    //lb.AddFilter("Microsoft.SemanticKernel", LogLevel.Trace);
    lb.AddSimpleConsole(opt => { 
        opt.TimestampFormat = "[hh:mm:ss.fff] "; 
        opt.SingleLine = true;});
});

// プラグインの組み込み
builder.Plugins.AddFromType<MyPlugin>("myplugin");

### まずは直接呼び出すパターン

実用することはあまりないと思うが動作確認として

In [5]:
var kernel = builder.Build();

// 現在時刻を取得するプラグイン
var ret1 = await kernel.InvokeAsync<DateTimeOffset>("myplugin", "get_today");
ret1.Display();

// 天気を取得するプラグインを呼び出す
var args = new KernelArguments(){
    {"date", DateTimeOffset.UtcNow.AddDays(1)}
};
var ret2 = await kernel.InvokeAsync<string>("myplugin", "get_weather", args);
ret2.Display();


[12:24:27.858] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today invoking.
[12:24:27.861] info: Submission#2.MyPlugin[0] getting today
[12:24:27.862] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today succeeded.
[12:24:27.866] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today completed. Duration: 0.003047s


2024-11-12 03:24:27Z

[12:24:27.870] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_weather invoking.
[12:24:27.871] info: Submission#2.MyPlugin[0] getting weather on 11/13/2024 03:24:27 +00:00
[12:24:27.871] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_weather succeeded.
[12:24:27.871] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_weather completed. Duration: 0.0014962s


雨

### テンプレートエンジンを使用して呼び出す

In [17]:
var kernel = builder.Build();

var promptText = """
小学生らしい文体で指定された日付の日記を書いてください。

今日の日付 : {{myplugin.get_today}}
""";

var ret3 = await kernel.InvokePromptAsync(promptText);
ret3.GetValue<string>().Display();

[12:38:46.476] info: Microsoft.SemanticKernel.KernelFunction[0] Function (null)-InvokePromptAsync_187bc6e6b6364049a4c07789468856a1 invoking.
[12:38:46.477] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today invoking.
[12:38:46.477] info: Submission#2.MyPlugin[0] getting today
[12:38:46.477] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today succeeded.
[12:38:46.477] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today completed. Duration: 2.08E-05s
[12:39:34.609] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 46. Completion tokens: 404. Total tokens: 450.


2024年11月12日　火曜日

今日は学校が休みの日だったので、朝はゆっくり寝ていました。おひさまが明るくて、お部屋の中がぽかぽかでした。起きてから、まずは朝ごはんを食べました。ママが作ってくれたトーストと、ボクの好きなイチゴジャムをぬりました。すごくおいしかったです。

朝ごはんのあとは、近くの公園に遊びに行きました。秋の葉っぱがいっぱい落ちていて、友だちといっしょにかけっこしたり、葉っぱを集めて山を作ったりしました。なんだか、かさかさした音が楽しかったです。

お昼ごはんは、おうちに帰ってからスパゲッティを食べました。パパが作ってくれたミートソースが最高でした。デザートにリンゴも食べて、お腹いっぱいになりました。

午後はおうちで宿題をしました。算数の問題がちょっとむずかしかったけど、ママが手伝ってくれたので、なんとか全部解けました。そのあと、少しテレビを見てから、弟と一緒にブロックで遊びました。大きなお城ができて、大満足でした。

夕ご飯のあと、お風呂に入ってさっぱりして、もう眠たくなってきました。明日は学校があるので、今日のうちにランドセルの用意をしておこうと思います。

今日はとっても楽しい一日だったので、いい夢が見られそうです。また明日も楽しい一日になりますように。

おやすみなさい。

[12:39:34.609] info: Microsoft.SemanticKernel.KernelFunction[0] Function (null)-InvokePromptAsync_187bc6e6b6364049a4c07789468856a1 succeeded.
[12:39:34.609] info: Microsoft.SemanticKernel.KernelFunction[0] Function (null)-InvokePromptAsync_187bc6e6b6364049a4c07789468856a1 completed. Duration: 48.1325867s


### Function Calling を使用してプラグイン呼び出しの必要性を判定させる

In [7]:
#pragma warning disable SKEXP0001

// プラグインの自動判定
var execSetting = new PromptExecutionSettings(){
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

#### Chat Completion を使う

In [18]:
using Microsoft.SemanticKernel.ChatCompletion;

// チャット履歴を作成
var history = new ChatHistory();
history.AddSystemMessage("あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。");
history.AddUserMessage("クリスマスまであと何日？");

// チャット補完サービスを利用
var kernel = builder.Build();
var ccsvc = kernel.GetRequiredService<IChatCompletionService>();
var ret5 = await ccsvc.GetChatMessageContentAsync(history, execSetting, kernel);
history.Add(ret5);
//history.Display();

// チャット履歴をダンプする(再利用するためにデリゲートにしておく)
var dumpChatHistory = (ChatHistory history) => {
    #pragma warning disable SKEXP0001
    foreach(var msg in history)
    {
        Console.WriteLine($"{msg.Role} : {msg.Content}");
        var functionCalling = FunctionCallContent.GetFunctionCalls(msg);
        if(functionCalling.Any())
        {
            foreach(var fc in functionCalling)
            {
                var funcArgs = fc.Arguments.Count() == 0 ? "" : fc.Arguments.Select(fa => fa.Value).Aggregate((x, y) => $"{x}, {y}");
                Console.WriteLine($"    function_calling: {fc.FunctionName} {funcArgs}");
            }
        }
    }
};

dumpChatHistory(history);


[12:39:44.860] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 127. Completion tokens: 12. Total tokens: 139.
[12:39:44.860] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today invoking.
[12:39:44.860] info: Submission#2.MyPlugin[0] getting today
[12:39:44.860] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today succeeded.
[12:39:44.860] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today completed. Duration: 1.37E-05s
[12:39:56.731] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 171. Completion tokens: 26. Total tokens: 197.
system : あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。
user : クリスマスまであと何日？
Assistant : 
    function_calling: get_today 
tool : "2024-11-12T03:39:44.8605962+00:00"
Assistant : 今日は2024年11月12日です。クリスマス（12月25日）まであと43日あります。


#### 複数回のプラグイン呼び出し

In [19]:
using Microsoft.SemanticKernel.ChatCompletion;

#pragma warning disable SKEXP0001

// チャット履歴を作成
var history = new ChatHistory();
history.AddSystemMessage("あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。");
history.AddUserMessage("明日の天気を教えて。明日の日付と服装の注意も教えて。");

// チャット補完サービスを利用
var kernel = builder.Build();
var ccsvc = kernel.GetRequiredService<IChatCompletionService>();
var ret5 = await ccsvc.GetChatMessageContentAsync(history, execSetting, kernel);
history.Add(ret5);
//history.Display();
dumpChatHistory(history);


[12:40:13.526] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 140. Completion tokens: 12. Total tokens: 152.
[12:40:13.526] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today invoking.
[12:40:13.526] info: Submission#2.MyPlugin[0] getting today
[12:40:13.526] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today succeeded.
[12:40:13.526] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today completed. Duration: 1.39E-05s
[12:40:28.642] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 184. Completion tokens: 51. Total tokens: 235.
[12:40:28.642] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_weather invoking.
[12:40:28.642] info: Submission#2.MyPlugin[0] getting weather on 11/13/2024 00:00:00 +00:00
[12:40:28.642] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-

In [15]:
using Microsoft.SemanticKernel.ChatCompletion;

#pragma warning disable SKEXP0001

// チャット履歴を作成
var history = new ChatHistory();
history.AddSystemMessage("あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。");
history.AddUserMessage("過去10年でクリスマスに雪が降った年を教えて");

// チャット補完サービスを利用
var kernel = builder.Build();
var ccsvc = kernel.GetRequiredService<IChatCompletionService>();
var ret6 = await ccsvc.GetChatMessageContentAsync(history, execSetting, kernel);
history.Add(ret6);

// Microsoft.DotNet.Interactive.Formatting.Formatter.ListExpansionLimit = 30;
// history.Display();
dumpChatHistory(history);

[12:34:43.648] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 136. Completion tokens: 12. Total tokens: 148.
[12:34:43.649] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today invoking.
[12:34:43.649] info: Submission#2.MyPlugin[0] getting today
[12:34:43.649] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today succeeded.
[12:34:43.649] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today completed. Duration: 1.78E-05s
[12:35:13.407] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 180. Completion tokens: 226. Total tokens: 406.
[12:35:13.408] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_weather invoking.
[12:35:13.408] info: Submission#2.MyPlugin[0] getting weather on 12/25/2013 00:00:00 +09:00
[12:35:13.408] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin